## Installing Sentence Transsformer and other models/frameworks

In [1]:
# !pip install sentence_transformers

# Kindly add all your installations and versions if any in this cell.

## Importing necessary libraries. 
In the final version all imports should be stricly enlisted here.

In [2]:
import pandas as pd
import numpy as np
import spacy
from scipy import stats
from sklearn import linear_model

import warnings
warnings.filterwarnings("ignore")

from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

import torch 
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

import fasttext.util


2023-01-04 23:18:42.066649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 23:18:42.295584: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-04 23:18:42.440029: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-04 23:18:43.540565: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

## Load dataset: 7 marks
1 Download and unzip the dataset from this link http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz  **1 mark**

2 Complete the code in `read_sts_csv()`. **4.5 marks**

3 Create 3 dataframes one each for train, dev and val and print their final shapes. **1.5 marks**

In [3]:
def read_sts_csv(dataset_type="train", columns=['source', 'type', 'year', 'id', 'score', 'sent_a', 'sent_b']):
    path = "/home/ayush/NLP_Assignments/A3_Ayush_Jahnvi/stsbenchmark/" +"sts-"+ dataset_type + ".csv"
    return pd.read_csv(path, sep='\\t', on_bad_lines='skip',names=columns)


df_train = read_sts_csv("train")
df_test = read_sts_csv("test")
df_dev = read_sts_csv("dev")

# df_<dataset_type> = read_sts_csv(dataset_type) 
# create the train, dev and test dataframes

In [4]:
#shapes of the 3 datasets
print(df_train.shape)
print(df_dev.shape)
print(df_test.shape)


(5552, 7)
(1478, 7)
(1095, 7)


In [5]:
df_train.head()

,source,type,year,id,score,sent_a,sent_b
0,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.
1,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.
2,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.
4,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.


## Hyperparameters: 5 Marks
Update this cell with you choosen parameters except, NUM_EPOCHS

In [3]:

NON_CONEXTUAL_MODEL_TYPE = fasttext.load_model('cc.en.300.bin')

CONEXTUAL_MODEL_TYPE = models.Transformer('distilroberta-base')

HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL = SentenceTransformer('all-MiniLM-L6-v2')# USE THE HUGGAING FACE VERSION OF SENTENCE_TRANSFORMER_TYPE
# INPUT_PATH = <INPUT_FOLDER_PATH>
BATCH_SIZE = 16
OUT_DIM_DENSE = 256
NUM_EPOCHS = 3 ## THIS IS FIXED DO NOT CHANGE

# You are free to add your own hyperparameters as well.

ValueError: cc.en.300.bin cannot be opened for loading!

## CONFIGURATION 1: Non-contextual Embeddings + ML Regression: 8 marks
1 Load the non-contextual embedding model in variable `non_cont_model1`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model1()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model1`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and dev set predictions using trained `model1`. **1.5 mark**



In [9]:
def get_feature_model1(data_frame):
    """
    Input a data frame and return the embedding vectors for the each sentence column using non_cont_model1,
    Return 2 matrices each of shape (#_samples, #size_of_word_emb).
    """
    df_sent = data_frame[["sent_a", "sent_b"]]
    list_A, list_B = [], []
    for row in range(len(df_sent)):
    list_A.append(NON_CONEXTUAL_MODEL_TYPE.get_sentence_vector(df_sent.loc[row, "sent_a"]))
    list_B.append(NON_CONEXTUAL_MODEL_TYPE.get_sentence_vector(df_sent.loc[row, "sent_b"]))

    return (np.array(list_A), np.array(list_B))  

 
## Non contextual language model
feature_1_train, feature_2_train = get_feature_model1(df_train)
X_train_tup, Ytrain = np.array(list(zip(feature_1_train, feature_2_train))), np.array(df_train["score"])
X_train = np.reshape(X_train_tup, (len(df_train), 600))

#word embedding of each sentence is a 300 sized vector, final df of 2 sentences containing 600 features

In [10]:
# Initiate a regression model and train it.
import numpy as np
from scipy import stats
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
clf = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
clf.fit(X_train, Ytrain)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.2))])

In [11]:
# # Print spearman correlation on the predicted output of the dev set.
feature_1_dev, feature_2_dev = get_feature_model1(df_dev)
X_dev_tup, Ydev = np.array(list(zip(feature_1_dev, feature_2_dev))), np.array(df_dev["score"])
X_dev = np.reshape(X_dev_tup, (len(df_dev), 600))
Y_pred_dev = clf.predict(X_dev)
spear_dev = stats.spearmanr(Ydev, Y_pred_dev)
print("Spearman Score on dev: " + str(spear_dev[0]))

Spearman Score on dev: 0.28387182507452213


In [12]:
# # Print spearman correlation on the predicted output of the test sets.
feature_1_test, feature_2_test = get_feature_model1(df_test)
X_test_tup, Ytest = np.array(list(zip(feature_1_test, feature_2_test))), np.array(df_test["score"])
X_test = np.reshape(X_test_tup, (len(df_test), 600))
Y_pred_test = clf.predict(X_test)
spear_test = stats.spearmanr(Ytest, Y_pred_test)
print("Spearman Score on test: " + str(spear_test[0]))

Spearman Score on test: 0.47763114660927947


## CONFIGURATION 2: Contextual Embeddings + ML Regression: 7 marks
1 Load the contextual embedding model in variable `non_cont_model2`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model2()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model2`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model2`. **1.5 mark**

Useful references: https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [13]:
def get_feature_model2(data_frame: pd.DataFrame):
  """
  Input a data frame and return the embedding vectors for the each sentence column using model2,
  Return 2 matrices each of shape (#_samples, #size_of_word_emb).
  """
    df_sent = data_frame[["sent_a", "sent_b"]]
    list_A, list_B = [], []
    sent_list_A = data_frame["sent_a"].to_list()
    sent_list_B = data_frame["sent_b"].to_list()

    embeddings1 = HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL.encode(sent_list_A, convert_to_tensor=True)
    embeddings2 = HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL.encode(sent_list_B, convert_to_tensor=True)

    return (np.array(embeddings1), np.array(embeddings2))


# non_cont_model2
feature_1_train_cont, feature_2_train_cont = get_feature_model2(df_train)
X_train_cont_tup, Y_train_cont = np.array(list(zip(feature_1_train_cont, feature_2_train_cont))), np.array(df_train["score"])
# print(np.array(feature_1_train_cont).shape)
# print(feature_1_train_cont.shape)

In [14]:
## Support vector regressor model created
# output of sentence transformer is a 384 dimension vector for 1 sentence, 768 for both 
X_train_cont = np.reshape(X_train_cont_tup, (len(df_train), (384*2)))
clf_cont = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
clf.fit(X_train_cont, Y_train_cont)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.2))])

In [15]:
# print spearman score on dev set
feature_1_dev_cont, feature_2_dev_cont = get_feature_model2(df_dev)
X_dev_tup_cont, Ydev = np.array(list(zip(feature_1_dev_cont, feature_2_dev_cont))), np.array(df_dev["score"])
X_dev_cont = np.reshape(X_dev_tup_cont, (len(df_dev), 768))
Y_pred_dev_cont = clf.predict(X_dev_cont)
spear_dev = stats.spearmanr(Ydev, Y_pred_dev_cont)
print("Spearman Score on dev: " + str(spear_dev[0]))

Spearman Score on dev: 0.4694053543121098


In [16]:
# print spearman score on test set
feature_1_test_cont, feature_2_test_cont = get_feature_model2(df_test)
X_test_tup_cont, Ytest = np.array(list(zip(feature_1_test_cont, feature_2_test_cont))), np.array(df_test["score"])
X_test_cont = np.reshape(X_test_tup_cont, (len(df_test), 768))
Y_pred_test_cont = clf.predict(X_test_cont)
spear_test = stats.spearmanr(Ytest, Y_pred_test_cont)
print("Spearman Score on test: " + str(spear_test[0]))

Spearman Score on test: 0.5767784080921666


## CONFIGURATION 3: Fine-Tune a Contextual Embeddings Model: 18 marks
1 Prepare data samples to be for the DL model to consume. Add the code in the `form_data()`. **4 marks**

3 Create the data loader, one each for train/dev/test data_input sample set obtained from `form_input_example()`. **1.5 marks**

4 Initiate `model3` consisting of **atleast** the following 3 components - `base_LM`, a `pooling_layer` and a `dense_layer`. Use appropriate activation function in dense. **Atleast** one layer of `base_LM` should be set to trainable. **5 marks**

6 Initiate the `loss`. **0.5 marks**

7 Fit the `model3`. Use `NUM_EPOCHS = 2`. **MAX_NUM_EPOCHS allowed will be 3**. **2 marks** 

8 Complete the `get_model_predicts()` to obtain predicted scores for input sentence pairs. **3.5 marks** 

9 Print the correlation scores on the dev and test set predictions. **1.5 mark**

Useful References: https://huggingface.co/blog/how-to-train-sentence-transformers 

In [15]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses
from tqdm.auto import tqdm
from sentence_transformers import models
from torch import nn
from datetime import datetime
import math
from sklearn.metrics.pairwise import paired_cosine_distances
from scipy.stats import  spearmanr


train_samples = []
dev_samples = []
test_samples = []


def form_data(data_frame, type):
  """
  Input a data frame and return the dataloder.
  """

  for row in range(len(data_frame)):
    score = float(data_frame.loc[row, "score"])/5  #normalise
    inp_ex = InputExample(texts = [data_frame.loc[row, "sent_a"], data_frame.loc[row, "sent_b"]], label=score)
    if type == 'train':
        train_samples.append(inp_ex)
    if type == 'test':
        test_samples.append(inp_ex)
    if type == 'dev':
        dev_samples.append(inp_ex)
    if type == 'train':
    return DataLoader(train_samples, shuffle=True, batch_size=BATCH_SIZE)
  else:
    return None



def get_model_predicts(train_example, trained_model):
  """
    Input the dataset list and return a list of cosine similarity scores. Use the fitted final_trainable_model for obtaining encodings.
    """
    sentences1 = []
    sentences2 = []
    scores = []

    for example in train_example:
        sentences1.append(example.texts[0])
        sentences2.append(example.texts[1])
        scores.append(example.label)


    embedding1 = trained_model.encode(sentences1, batch_size=BATCH_SIZE, convert_to_numpy=True)
    embedding2 = trained_model.encode(sentences2, batch_size=BATCH_SIZE, convert_to_numpy=True)



    cosine_scores = 1 - (paired_cosine_distances(embedding1, embedding2))


    return cosine_scores

In [16]:
# load the data
dataloader_train = form_data(df_train, 'train')
form_data(df_test, 'test')
form_data(df_dev, 'dev')

In [11]:
# model intialization

#pooling layer creates sentence embeddings from word embeddings by clubbing them
layer_pooling = models.Pooling(CONEXTUAL_MODEL_TYPE.get_word_embedding_dimension(), pooling_mode_mean_tokens=True, 
                               pooling_mode_cls_token=False, pooling_mode_max_tokens=False)

#adding a new layer to the output of the output of pooling helps learn task specific features
layer_dense = models.Dense(in_features=layer_pooling.get_sentence_embedding_dimension(), out_features=OUT_DIM_DENSE, activation_function=nn.Tanh())

model3 = SentenceTransformer(modules=[CONEXTUAL_MODEL_TYPE, layer_pooling, layer_dense])

#calculate cosine similarity loss
loss = losses.CosineSimilarityLoss(model = model3)

In [13]:
# fit the model
model3.fit(train_objectives=[(dataloader_train, loss)], epochs=NUM_EPOCHS)

Epoch: 100%|██████████| 3/3 [01:40<00:00, 33.57s/it]


In [17]:
# prediction of spearman score over train sset
cosine_scores = get_model_predicts(train_samples, model3)
spearman_scores, _ = spearmanr(df_train['score'], cosine_scores)
print(spearman_scores)

0.8212583479361734


In [18]:
# prediction of spearman score over dev sset
cosine_scores = get_model_predicts(dev_samples, model3)
spearman_scores, _ = spearmanr(df_dev['score'], cosine_scores)
print(spearman_scores)

0.8572441472850028


In [19]:
# prediction of spearman score over test sset
cosine_scores = get_model_predicts(test_samples, model3)
spearman_scores, _ = spearmanr(df_test['score'], cosine_scores)
print(spearman_scores)

0.8182538548138503


In [20]:
# for idx, (name, param) in enumerate(model.named_parameters()):
#     if idx <= 83:
#         param.requires_grad = False


# for idx, (name, param) in enumerate(model3.named_parameters()):
#     if param.requires_grad == True:
#         print(name)

0.auto_model.embeddings.word_embeddings.weight
0.auto_model.embeddings.position_embeddings.weight
0.auto_model.embeddings.token_type_embeddings.weight
0.auto_model.embeddings.LayerNorm.weight
0.auto_model.embeddings.LayerNorm.bias
0.auto_model.encoder.layer.0.attention.self.query.weight
0.auto_model.encoder.layer.0.attention.self.query.bias
0.auto_model.encoder.layer.0.attention.self.key.weight
0.auto_model.encoder.layer.0.attention.self.key.bias
0.auto_model.encoder.layer.0.attention.self.value.weight
0.auto_model.encoder.layer.0.attention.self.value.bias
0.auto_model.encoder.layer.0.attention.output.dense.weight
0.auto_model.encoder.layer.0.attention.output.dense.bias
0.auto_model.encoder.layer.0.attention.output.LayerNorm.weight
0.auto_model.encoder.layer.0.attention.output.LayerNorm.bias
0.auto_model.encoder.layer.0.intermediate.dense.weight
0.auto_model.encoder.layer.0.intermediate.dense.bias
0.auto_model.encoder.layer.0.output.dense.weight
0.auto_model.encoder.layer.0.output.dens